In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Logistic Regression with `autograd`

In [ ]:
import autograd.numpy as np
import autograd

We need to define the sigmoid function $S(t) := \large \frac{1}{1 + \exp(-t)}$.

In [ ]:
def sigmoid(t):
    return 1.0 / (1.0 + np.exp(-t))

The function `logSigmoid` computes the natural logarithm of the sigmoid function.  The implementation takes care of preventing *overflows* that would occur in a naive implementation for t < -100.

In [ ]:
def logSigmoid(t):
    if t > -100:
        return -np.log(1.0 + np.exp(-t))
    else:
        return t

In [ ]:
import csv

In [ ]:
with open('exam.csv') as file:
    reader = csv.reader(file, delimiter=',')
    count  = 0  # line count
    Pass   = []
    Hours  = []
    for row in reader:
        if count != 0:  # skip header
            Pass .append(float(row[0]))
            Hours.append(float(row[1]))
        count += 1

In [ ]:
y = np.array(Pass)
y = 2 * y - 1
x = np.array(Hours)

Given a feature matrix `X` and a vector `y` of classification outputs, the *log-likelihood function* $\texttt{ll}(\textbf{X}, \textbf{y},\textbf{w})$ is mathematically defined as follows:
$$\ell\ell(\mathbf{X},\mathbf{y},\mathbf{w}) = 
 \sum\limits_{i=1}^N \ln\Bigl(S\bigl(y_i \cdot(\mathbf{x}_i \cdot \mathbf{w})\bigr)\Bigr) =
 \sum\limits_{i=1}^N L\bigl(y_i \cdot(\mathbf{x}_i \cdot \mathbf{w})\bigr)
$$
The value of the *log-likelihood function* is interpreted as the logarithm of the probability that our model of the classifier predicts the observed values $y_i$ when the features are given by the vector $\textbf{x}_i$ for all $i\in\{1,\cdots,N\}$.

The arguments $\textbf{X}$, $\textbf{y}$, and $\textbf{w}$ are interpreted as follows:
* $\textbf{X}$ is the feature matrix, $\textbf{X}[i]$ is the $i$-th feature vector, i.e we have
  $\textbf{X}[i] = \textbf{x}_i$ if we regard $\textbf{x}_i$ as a row vector.
         
  Furthermore, it is assumed that $\textbf{X}[i][0]$ is 1.0 for all $i$.  
  Hence we have a feature that is constant for all examples.
* $\textbf{y}$ is the output vector, $\textbf{y}[i] \in \{-1,+1\}$ for all $i$.
* $\textbf{w}$ is the weight vector.

In [ ]:
def ll(w):
    𝛼, 𝛽   = w 
    result = 0.0
    n      = len(x)
    for i in range(n):
        result = result + logSigmoid(y[i] * (𝛼 * x[i] + 𝛽))
    return result

In [ ]:
gradLL = autograd.grad(ll)

In [ ]:
ll(np.array([0.0, 0.0]))

In [ ]:
gradLL(np.array([0.0, 0.0]))

In [ ]:
def findMaximum(f, gradF, start, eps):
    x     = start
    fx    = f(x)
    alpha = 0.1   # learning rate
    cnt   = 0     # number of iterations
    for k in range(500):
        cnt += 1
        xOld, fOld = x, fx
        x  += alpha * gradF(x)
        fx  = f(x)        
        if fx <= fOld:    # f didn't increased, learning rate is too high
            alpha *= 0.5  # decrease the learning rate
            x, fx = xOld, fOld    # reset x
            continue
        else:             # f has increased
            alpha *= 1.2  # increase the learning rate
    return x

In [ ]:
start   = np.array([0.0, 0.0])
eps     = 10 ** -5
gamma, beta = findMaximum(ll, gradLL, start, eps)
print(f'model: P(pass|hours) = S({beta} + {gamma} * hours)')

Let us plot this function together with the data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn           as sns

In [ ]:
plt.figure(figsize=(15, 9))
sns.set_style('whitegrid')
plt.title('Pass/Fail vs. Hours of Study')
H = np.arange(0.0, 6.0, 0.05)
P = sigmoid(beta + gamma * H)
sns.lineplot(x=H, y=P, color='r')
plt.axvline(x=0.0, c='k')
plt.axhline(y=0.0, c='k')
plt.xlabel('Hours of Study')
plt.ylabel('Probability of Passing the Exam')
plt.xticks(np.arange(0.0, 6.0, step=0.5))
plt.yticks(np.arange(-0.0, 1.01, step=0.1))
plt.scatter(x, (y + 1) / 2, color='b')
plt.savefig('exam-probability.pdf')